In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import ipdb

In [87]:
class Node(object):
    def __init__(self, typ, val):
        self.type = typ
        self.value = val
        self.left = None
        self.right = None
    def __str__(self):
        return '%s(%s) -> [%s(%s), %s(%s)]'%(self.type, self.value, self.left.type, self.left.value, self.right.type, self.right.value)
    def setLeft(self, L):
        self.left = L
    def setRight(self, R):
        self.right = R

In [102]:
def BuildTree(currNode, currDepth, maxDepth, nodes, nodeType, nodeLens):
    #ipdb.set_trace()
    # exit if too deep or at a leaf
    if (currDepth > maxDepth) or (currNode.type != 'ops'):
        return currNode
    # randomly generate the left node
    nodeTypeL = nodeTypes[np.random.randint(3)]
    nodeValuL = nodes[nodeTypeL][np.random.randint(nodeLens[nodeTypeL])]
    currNode.setLeft(BuildTree(Node(nodeTypeL, nodeValuL), currDepth+1, maxDepth, nodes, nodeType, nodeLens))

    # randomly generate the right node
    nodeTypeR = nodeTypes[np.random.randint(3)]
    nodeValuR = nodes[nodeTypeR][np.random.randint(nodeLens[nodeTypeR])]
    currNode.setRight(BuildTree(Node(nodeTypeR, nodeValuR), currDepth+1, maxDepth, nodes, nodeType, nodeLens))
    
    return currNode

In [85]:
ops = ['+', '-', '*', '\\', '^', 'max', 'min']
feats = ['x%d'%i for i in range(5)]
consts = list(range(1, 11))

nodes = {'ops':ops, 'feats':feats, 'consts':consts}
nodeLens = {'ops':len(ops), 'feats':len(feats), 'consts':len(consts)}
nodeTypes = list(nodes.keys())

In [103]:
''' randomly generate a tree '''
# set seed
nw = dt.datetime.now()
randSeed = 175942#nw.hour*10000 + nw.minute*100 + nw.second
print('Random seed = %d'%randSeed)
np.random.seed(randSeed)

# set the depth
maxDepth = 10
depth = np.random.randint(low=1, high=maxDepth+1)

# build the tree, starting from the top node
nodeType = nodeTypes[np.random.randint(3)]
nodeValu = nodes[nodeType][np.random.randint(nodeLens[nodeType])]
print('%s(%s)'%(nodeType, nodeValu))
thisTree = BuildTree(Node(nodeType, nodeValu), 0, 3, nodes, nodeType, nodeLens)

Random seed = 175942
ops(^)


In [109]:
print(thisTree)

ops(^) -> [ops(-), feats(x0)]
